In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Importamos dataset

In [3]:
churn = pd.read_csv('/content/drive/My Drive/4- Data Science/Clases UBA/KNN | Regresion Logistica | CV/Churn_Modelling.csv')

In [4]:
churn.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# Analisis exploratorio de los datos

## Dataset Churn

In [5]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [7]:
churn.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [13]:
churn['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [14]:
churn['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [15]:
# borramos las columnas categóricas y aquellas numericas que no hacen sentido y generan ruido en el modelo.

churn.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

## OneHotEncoding

In [61]:
# guardamos los atributos en un NumPy Array, ya que es el formato en el cual nos sklearn recibe los datos

X = churn.loc[:, churn.columns != 'Exited'].values
y = churn['Exited'].values

Para profundizar en el concepto, les dejamos la siguiente publicación: 

https://medium.com/@contactsunny/label-encoder-vs-one-hot-encoder-in-machine-learning-3fc273365621

In [62]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [63]:
# importamos la libreria para poder realizar el One Hot Encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Como podemos ver, las columnas que tienen los valores categoricos son la segunda y la tercera (Geography y Gender)

cl = ColumnTransformer(transformers = [('OHE', OneHotEncoder(drop='first'), [1,2])], remainder='passthrough')

In [64]:
X = cl.fit_transform(X)

In [66]:
X[1:10]

array([[0.0, 1.0, 0.0, 608, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [0.0, 0.0, 0.0, 502, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [0.0, 0.0, 0.0, 699, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [0.0, 1.0, 0.0, 850, 43, 2, 125510.82, 1, 1, 1, 79084.1],
       [0.0, 1.0, 1.0, 645, 44, 8, 113755.78, 2, 1, 0, 149756.71],
       [0.0, 0.0, 1.0, 822, 50, 7, 0.0, 2, 1, 1, 10062.8],
       [1.0, 0.0, 0.0, 376, 29, 4, 115046.74, 4, 1, 0, 119346.88],
       [0.0, 0.0, 1.0, 501, 44, 4, 142051.07, 2, 0, 1, 74940.5],
       [0.0, 0.0, 1.0, 684, 27, 2, 134603.88, 1, 1, 1, 71725.73]],
      dtype=object)

# Train/Test Spit

Vamos a necesitar separar nuestros datos en un conjunto para entrenar al modelo y otro para probarlo, esto nos va a servir para poder verificar la performance de nuestro modelo con datos que no conoce.

In [52]:
from sklearn.model_selection import train_test_split  # una vez hacerlo a mano (con sample) así entienden bien de qué se trata

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.25, 
                                                    random_state = 15,
                                                    stratify = y)

# Regresión Logística

Para trabajar con regresión logística, vamos a utilizar de la libreria de Sklearn del apartado de modelos lineales, la clase LogisticRegression. Para importarla simplemente la llamamos de la siguiente manera:

    from sklearn.linear_model import LogisticRegression

A continuación veremos como inicializar una instancia de dicha clase, como entrenar nuestro modelo y predecir nuevos valores.

Los Hiperparametros que vamos a estudiar de este modelo son:

- C: relacionado a la magnitud de regularización aplicada al modelo.
- penalty: que tipo de regularización sera utilizada, por defecto l2.

A tener en cuenta:

Al ser un modelo lineal, el resultado van a ser coeficientes que multiplican a los atributos y una ordenada al origen, pero como determinamos que clase es la correspondiente?

Por el signo, si es postivo predice una clase, si es negativo la otra clase.

IMPORTANTE!

El modo en el cual operamos con casi todos los modelos de sklearn es practicamente el mismo, con lo cual seguiremos los mismos pasos que en el modelo de KNN.

In [54]:
from sklearn.linear_model import LogisticRegression

In [55]:
lr = LogisticRegression()

lr.fit(X_train, y_train)

print('Los resultados son: \n')
print('El accuracy de prueba es de: {}'.format(lr.score(X_train, y_train)))
print('El accuracy de test es de: {}'.format(lr.score(X_test, y_test)))

Los resultados son: 

El accuracy de prueba es de: 0.7908
El accuracy de test es de: 0.79


## Interpretación de datos

Ahora interpretemos los datos que nos ofrecen los métodos de predict y predict_proba

El primero, es simplemente que clase se predice teniendo en cuenta que el corte de clasificación se encuentra en p = 0.5.

El segundo, nos brinda un arreglo con las probabilidades de cada una de las clases.

Investiguemos con algun ejemplo estos valores!

In [56]:
lr.predict(X_test[:2])

array([0, 0])

In [57]:
lr.predict_proba(X_test[:2])

array([[0.79238302, 0.20761698],
       [0.87903645, 0.12096355]])

**Cambio de umbral de predicción:**

Ahora bien, que pasa si queremos cambiar el umbral con el cual determinamos la clase correcta?

Facil! Accedemos al valor de la clase que queremos someter a otro umbral de todos los arreglos y lo sometemos a una comprobación logica. El resultado sera un arreglo con booleanos (que se transforman en 1 y 0)

**IMPORTANTE:** si queremos comparar correctamente las clases, deberíamos aplicarle la condición logica a la clase que finalmente llamamos 1
sino, nos estaría dando la clase opuesta!

In [68]:
umbral_25 = lr.predict_proba(X_test)[:, 1] > 0.25

In [69]:
# por ultimo volvemos a introducir el concepto de Confusion maxtrix visto en la teoria

from sklearn.metrics import classification_report, confusion_matrix

confusion_matrix(y_test, lr.predict(X_test))

array([[1949,   42],
       [ 483,   26]])

In [70]:
# ahora veamos como varia la matriz de confusion con el umbral de aceptacion en 0.25

confusion_matrix(y_test, umbral_25)

array([[1495,  496],
       [ 265,  244]])

In [60]:
# ahora vamos a ver el reporte de clasificación para el umbral 0.5

print(classification_report(y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.80      0.98      0.88      1991
           1       0.38      0.05      0.09       509

    accuracy                           0.79      2500
   macro avg       0.59      0.51      0.49      2500
weighted avg       0.72      0.79      0.72      2500



In [71]:
# ahora vamos a ver el reporte de clasificación para el umbral 0.25

print(classification_report(y_test, umbral_25))

              precision    recall  f1-score   support

           0       0.85      0.75      0.80      1991
           1       0.33      0.48      0.39       509

    accuracy                           0.70      2500
   macro avg       0.59      0.62      0.59      2500
weighted avg       0.74      0.70      0.71      2500



# Cross validation

**Pasos para la composición del código**

- Importar las librerias (LogisticRegression ya la tenemos importada asi que solo cross_val_score)

      from sklearn.model_selection import cross_val_score

- Inicializar el modelo (como siempre)
      lr = LogisticRegression()
- Inicializar cross validation
      cv_score = cross_val_score(modelo a utilizar, X, y, cv = cantidad de splits)

In [ ]:
from sklearn.model_selection import cross_val_score

lr = LogisticRegression()

cv_score = cross_val_score(lr, X, y, cv=5)

In [ ]:
cv_score

array([0.7915, 0.791 , 0.7935, 0.789 , 0.7885])

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (cv_score.mean(), cv_score.std() * 2))

Accuracy: 0.79 (+/- 0.00)
